In [1]:
import os
import sys
import pandas as pd
pd.set_option('display.max_rows', None)  # Set to None to display all rows
import urllib
import psycopg2
from psycopg2.extras import execute_values
from datetime import datetime, timedelta, timezone

# Add project root to sys.path
current_dir = os.getcwd()
project_root = os.path.abspath(os.path.join(current_dir, '..'))
if project_root not in sys.path:
    sys.path.append(project_root)

# Verify sys.path
print("Current sys.path:", sys.path)

from src.db.base_utils import get_db_connection
from src.db.the_odds_db_utils import *

Current sys.path: ['C:\\Users\\H-100\\AppData\\Local\\Programs\\Python\\Python310\\python310.zip', 'C:\\Users\\H-100\\AppData\\Local\\Programs\\Python\\Python310\\DLLs', 'C:\\Users\\H-100\\AppData\\Local\\Programs\\Python\\Python310\\lib', 'C:\\Users\\H-100\\AppData\\Local\\Programs\\Python\\Python310', 'c:\\Users\\H-100\\Desktop\\python\\accurateshothelper\\.venv', '', 'c:\\Users\\H-100\\Desktop\\python\\accurateshothelper\\.venv\\lib\\site-packages', 'c:\\Users\\H-100\\Desktop\\python\\accurateshothelper\\.venv\\lib\\site-packages\\win32', 'c:\\Users\\H-100\\Desktop\\python\\accurateshothelper\\.venv\\lib\\site-packages\\win32\\lib', 'c:\\Users\\H-100\\Desktop\\python\\accurateshothelper\\.venv\\lib\\site-packages\\Pythonwin', 'c:\\Users\\H-100\\Desktop\\python\\accurateshothelper']


c:\Users\H-100\Desktop\python\accurateshothelper\.venv\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
# Today's date (EST)
(datetime.now() - timedelta(hours=6)).strftime('%Y-%m-%d')

'2025-03-31'

In [3]:
# fetch_and_store_nhl_games('2025-01-30', enable_logging=True)

In [4]:
# get_nhl_events_from_db('2025-01-06', enable_logging=True)

In [5]:
get_mismatched_game_ids_with_details(enable_logging=True)


INFO:root:Comparing distinct game_ids in nhl_game_info and nhl_player_sog_odds tables.
INFO:src.db.base_utils:Attempting connection with: host=localhost, port=5432, dbname=the_odds_db, user=postgres
INFO:src.db.base_utils:Successfully established database connection
INFO:root:Game IDs only in nhl_game_info: [('fa2857f75712aa8583ce8d7681eed9db', 'Nashville Predators', 'Minnesota Wild', datetime.datetime(2024, 2, 29, 19, 0, tzinfo=datetime.timezone(datetime.timedelta(days=-1, seconds=64800)))), ('8e6022b26143a4f186ea62572d6a5e18', 'Chicago Blackhawks', 'Colorado Avalanche', datetime.datetime(2024, 2, 29, 20, 0, tzinfo=datetime.timezone(datetime.timedelta(days=-1, seconds=64800)))), ('6ce908f4d4ac2950b9728caeb40ce0a6', 'Vancouver Canucks', 'Los Angeles Kings', datetime.datetime(2024, 2, 29, 21, 0, tzinfo=datetime.timezone(datetime.timedelta(days=-1, seconds=64800)))), ('ae13bac2d5e4f66e4bd30d009737ee8f', 'Seattle Kraken', 'Pittsburgh Penguins', datetime.datetime(2024, 2, 29, 21, 0, tzinfo

{'only_in_game_info': [{'game_id': 'fa2857f75712aa8583ce8d7681eed9db',
   'away_team': 'Minnesota Wild',
   'home_team': 'Nashville Predators',
   'commence_time': '2024-02-29'},
  {'game_id': '8e6022b26143a4f186ea62572d6a5e18',
   'away_team': 'Colorado Avalanche',
   'home_team': 'Chicago Blackhawks',
   'commence_time': '2024-02-29'},
  {'game_id': '6ce908f4d4ac2950b9728caeb40ce0a6',
   'away_team': 'Los Angeles Kings',
   'home_team': 'Vancouver Canucks',
   'commence_time': '2024-02-29'},
  {'game_id': 'ae13bac2d5e4f66e4bd30d009737ee8f',
   'away_team': 'Pittsburgh Penguins',
   'home_team': 'Seattle Kraken',
   'commence_time': '2024-02-29'},
  {'game_id': 'e9b33ddce549d92b2f1b04fc4dd20195',
   'away_team': 'Anaheim Ducks',
   'home_team': 'San Jose Sharks',
   'commence_time': '2024-02-29'},
  {'game_id': '366a884213a807c78c1234f0a6858687',
   'away_team': 'Arizona Coyotes',
   'home_team': 'Ottawa Senators',
   'commence_time': '2024-03-01'},
  {'game_id': 'efdad280e941b68c9305

In [6]:
# process_sog_markets('76d32822c0db9bfcc89357f081134d89', '2025-01-05', enable_logging=True)

In [7]:
# process_all_sog_markets('2025-01-25', enable_logging=True)

In [8]:
def process_all_sog_markets_for_month(year: int, month: int, enable_logging: bool = True) -> None:
    """
    Runs the `process_all_sog_markets` function for each day in the specified month.

    Args:
        year (int): The year of the month to process.
        month (int): The month to process (1-12).
        enable_logging (bool, optional): Enable logging. Defaults to True.
    """
    import calendar
    from datetime import datetime

    # Get the number of days in the specified month
    num_days = calendar.monthrange(year, month)[1]

    for day in range(1, num_days + 1):
        # Format the date as 'YYYY-MM-DD'
        date_str = datetime(year, month, day).strftime('%Y-%m-%d')
        
        # Log the current date being processed
        if enable_logging:
            print(f"Processing SOG markets for date: {date_str}")
        
        # Call the existing function to process SOG markets for the current date
        process_all_sog_markets(date_str, enable_logging=enable_logging)

In [9]:
# process_all_sog_markets_for_month(year= 2025, month= 2)

In [10]:
def process_all_sog_markets_for_date_range(start_date: str, end_date: str, enable_logging: bool = True) -> None:
    """
    Runs the `process_all_sog_markets` function for each date within the specified date range.
    
    Args:
        start_date (str): The start date in 'YYYY-MM-DD' format.
        end_date (str): The end date in 'YYYY-MM-DD' format.
        enable_logging (bool, optional): Enable logging. Defaults to True.
    """
    from datetime import datetime, timedelta

    # Convert string dates to datetime objects
    start_dt = datetime.strptime(start_date, '%Y-%m-%d')
    end_dt = datetime.strptime(end_date, '%Y-%m-%d')

    if start_dt > end_dt:
        raise ValueError("start_date must be on or before end_date")

    current_dt = start_dt
    while current_dt <= end_dt:
        date_str = current_dt.strftime('%Y-%m-%d')
        
        # Log the current date being processed
        if enable_logging:
            print(f"Processing SOG markets for date: {date_str}")
        
        # Call the existing function to process SOG markets for the current date
        process_all_sog_markets(date_str, enable_logging=enable_logging)
        
        # Move to the next day
        current_dt += timedelta(days=1)

# Example usage
# process_all_sog_markets_for_date_range(start_date='2024-04-01', end_date='2024-04-18')

In [11]:
def process_all_moneyline_markets_for_date_range(start_date: str, end_date: str, enable_logging: bool = True) -> None:
    """
    Runs the `process_all_moneyline_markets` function for each date within the specified date range.
    
    Args:
        start_date (str): The start date in 'YYYY-MM-DD' format.
        end_date (str): The end date in 'YYYY-MM-DD' format.
        enable_logging (bool, optional): Enable logging. Defaults to True.
    """
    from datetime import datetime, timedelta

    # Convert string dates to datetime objects
    start_dt = datetime.strptime(start_date, '%Y-%m-%d')
    end_dt = datetime.strptime(end_date, '%Y-%m-%d')

    if start_dt > end_dt:
        raise ValueError("start_date must be on or before end_date")

    current_dt = start_dt
    while current_dt <= end_dt:
        date_str = current_dt.strftime('%Y-%m-%d')
        
        # Log the current date being processed
        if enable_logging:
            print(f"Processing moneyline markets for date: {date_str}")
        
        # Call the existing function to process moneyline markets for the current date
        process_all_moneyline_markets(query_date=date_str, enable_logging=enable_logging)
        
        # Move to the next day
        current_dt += timedelta(days=1)

# Example usage
# process_all_moneyline_markets_for_date_range(start_date='2025-02-22', end_date='2025-02-28')

In [12]:
# process_all_saves_markets_date_range(start_date='2024-10-04', end_date='2024-10-31', enable_logging=True)

In [15]:
# fetch_and_store_mlb_games('2025-03-28', True)

# this doesn't work
# process_strikeout_markets("6e31e96d4868211d236d00d115a63bc8", enable_logging=True)

process_all_strikeout_markets('2025-03-28', enable_logging=True)

INFO:root:Processing all strikeout markets for date: 2025-03-28
INFO:root:Retrieving MLB events from DB for date: 2025-03-28
INFO:src.db.base_utils:Attempting connection with: host=localhost, port=5432, dbname=the_odds_db, user=postgres
INFO:src.db.base_utils:Successfully established database connection
INFO:root:Retrieved 9 events from the database for date 2025-03-28.
INFO:root:Processing game a9f23bc743f697d1776bbd97dca4ee1e scheduled for 2025-03-28T23:07:00Z
INFO:root:Processing strikeout markets for game: a9f23bc743f697d1776bbd97dca4ee1e
INFO:root:Using historical odds for date: 2025-03-28T23:07:00Z
INFO:src.db.base_utils:Attempting connection with: host=localhost, port=5432, dbname=the_odds_db, user=postgres
INFO:src.db.base_utils:Successfully established database connection
INFO:root:Inserted/Updated 36 records for game a9f23bc743f697d1776bbd97dca4ee1e
INFO:root:Completed processing strikeout markets for game a9f23bc743f697d1776bbd97dca4ee1e
INFO:root:Processing game e60ef7591f6